# PyTorch Quickstart with nbdev

> An examle of taking the PyTorch getting starting project and documenting with nbdev

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Quickstart
> This section runs through API for common tasks in machine learning

### Working with Data

PyTorch has two [`primitives to work with data`](https://pytorch.org/docs/stable/data.html>):
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from yaml import safe_load

params = safe_load(open("../params.yaml"))

from dvclive import Live

live = Live("../training")

PyTorch offers domain-specific libraries such as [`TorchText`]( <https://pytorch.org/text/stable/index.html>), [`TorchVision`](https://pytorch.org/vision/stable/index.html), and [`TorchAudio`](https://pytorch.org/audio/stable/index.html), all of which include datasets. For this tutorial, we  will be using a `TorchVision` dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like CIFAR, COCO ([full list here](https://pytorch.org/vision/stable/datasets.html)). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and ``target_transform`` to modify the samples and labels respectively.


Download training data from open datasets.

In [ ]:
training_data = datasets.FashionMNIST(
    root="../data",
    train=True,
    download=True,
    transform=ToTensor(),
)

Download test data from open datasets.

In [ ]:
test_data = datasets.FashionMNIST(
    root="../data",
    train=False,
    download=True,
    transform=ToTensor(),
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [ ]:
batch_size = 64

Create data loaders

In [ ]:
test_dataloader = DataLoader(test_data, batch_size=batch_size)
train_dataloader = DataLoader(training_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


### Creating Models

To define a neural network in PyTorch, we create a class that inherits from [`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). We define the layers of the network in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. To accelerate operations in the neural network, we move it to the GPU if available.

Get cpu or gpu device for training

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Define model

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Read more about [building neural networks in PyTorch](buildmodel_tutorial.html>)

### Optimizing Model

To train a model, we need a [`loss function`](https://pytorch.org/docs/stable/nn.html#loss-functions>) and an [`optimizer`](https://pytorch.org/docs/stable/optim.html)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=params["lr"])

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model's parameters.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    last_loss = 0.0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            last_loss = loss
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.


In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the accuracy increase and the loss decrease with every epoch.

In [ ]:
epochs = params["epochs"]
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    live.next_step()
print("Done!")

Epoch 1
-------------------------------
loss: 2.311246  [    0/60000]
loss: 2.306192  [ 6400/60000]
loss: 2.307190  [12800/60000]
loss: 2.303523  [19200/60000]
loss: 2.302620  [25600/60000]
loss: 2.310335  [32000/60000]
loss: 2.306589  [38400/60000]
loss: 2.312041  [44800/60000]
loss: 2.308785  [51200/60000]
loss: 2.304009  [57600/60000]
Test Error: 
 Accuracy: 6.4%, Avg loss: 2.305941 

Epoch 2
-------------------------------
loss: 2.311246  [    0/60000]
loss: 2.306192  [ 6400/60000]
loss: 2.307190  [12800/60000]
loss: 2.303523  [19200/60000]
loss: 2.302620  [25600/60000]
loss: 2.310335  [32000/60000]
loss: 2.306589  [38400/60000]
loss: 2.312041  [44800/60000]
loss: 2.308785  [51200/60000]
loss: 2.304009  [57600/60000]
Test Error: 
 Accuracy: 6.4%, Avg loss: 2.305941 

Epoch 3
-------------------------------
loss: 2.311246  [    0/60000]
loss: 2.306192  [ 6400/60000]
loss: 2.307190  [12800/60000]
loss: 2.303523  [19200/60000]
loss: 2.302620  [25600/60000]
loss: 2.310335  [32000/60000

### Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [ ]:
torch.save(model.state_dict(), "../model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()